# Neural Net From Scratch

### Imports

In [ ]:
import numpy as np

np.random.seed(1)

### Activation functions

In [ ]:
class Linear():
    @staticmethod
    def f(x):
        return x
    @staticmethod
    def df(x):
        return x

class Relu():
    @staticmethod
    def f(x):
        return x * (x > 0)
    @staticmethod
    def df(x):
        return 1 * (x > 0)

class Sigmoid():
    @staticmethod
    def f(x):
        return 1.0 / (1.0 + np.exp(-x))
    @staticmethod
    def df(x):
        return Sigmoid.f(x) * (1 - Sigmoid.f(x))

### Loss functions

In [513]:
class MSE():
  @staticmethod
  def f(target, output):
    return np.mean((target - output) **2)
  
  @staticmethod 
  def df(target, output):
    return 2 * (target - output) / np.size(output)

class Crossentropy():
  @staticmethod
  def f(target, output):
    return np.mean(-target * np.log(output) - (1 - target) * np.log(1 - output))
  
  @staticmethod 
  def df(target, output):
    return ((1 -  target) / (1 - output) - target/output) / np.size(target)  

### Dense layer

In [469]:
class Layer:
    def __init__(self, neurons, activation=Linear):
        self.neurons = neurons
        self.activation = activation
        self.weights = None
        self.biases = None
        self.lr = 0.02 # Snelheid waarmee de model traint -> staat nu hardcoded

    def initialize(self, prev_neurons):
        self.weights = np.random.normal(0.0, 1.0, (self.neurons, prev_neurons))
        self.biases = np.random.normal(size=(self.neurons, 1))

    def feed_forward(self, input_array):
        # Input en output opslaan -> zijn nodig voor gradient descent
        self.inputs = input_array 

        # Y = W * I + B (matrix * vector)
        self.output = self.activation.f(np.dot(self.weights, self.inputs) + self.biases) 
        return self.output

    def gradient_descent(self, errors):
        delta_weights = self.lr * np.dot((errors * self.activation.df(self.output)), self.inputs.T)
        delta_biases = self.lr * errors * self.activation.df(self.output)

        self.weights = np.add(self.weights, delta_weights)
        self.biases = np.add(self.biases, delta_biases)

### Neurale netwerk

In [553]:
class NeuralNetwork():
    def __init__(self):
        self.layers = []

    def add(self, layer):
        # Initialiseer de laag met weights en biases alleen als het niet de input laag is.
        if len(self.layers) > 0:
            prev_neurons = self.layers[-1].neurons
            layer.initialize(prev_neurons)
        
        self.layers.append(layer)
        

    def predict(self, input_array, transposed = False):
        
        output = input_array
        
        if not transposed:
            output = output.T

        for lay in self.layers[1:]:
            output = lay.feed_forward(output)
        return output

    def fit(self, X_values: np.ndarray ,y_true: np.ndarray, loss_function=MSE, epochs=10):

        for epoch in range(epochs):
            sum_errors = 0

            for x, y in zip(X_values, y_true):
                
                # feedforward
                prediction = self.predict(x.reshape(-1,1), transposed = True) # Reshape maakt ndmin 2 en transposed het al

                # print('prediction:')
                # print(prediction)
                # print('y')
                # print(y)


                sum_errors += loss_function.f(y, prediction) # houdt mse bij om het te visualiseren

                error = np.array(loss_function.df(y, prediction), ndmin=2) # gebruik afgeleide om weten of we de gewichten moeten verhogen of verlagen
                #error = y - prediction
                #sum_errors += cross_entropy(y, prediction[0][0]) # houdt mse bij om het te visualiseren
             
                
                # backward propagation
                # error = mse_derivative(y, prediction)

                for lay in reversed(self.layers[1:]):
                    lay.gradient_descent(error)
                    error = np.dot(lay.weights.T, error)           
            sum_errors /= len(X_values)

            print(f"Epoch {epoch+1}/{epochs}:")
            print(f"Error: {sum_errors}\n")
        print(f"\nFinished Training\n{'=' * 50}")



### Train

Getallen optellen

e.g. array van [0.23, 0.56] = 0.79

In [560]:
from random import random, seed

seed(42)

nn = NeuralNetwork()

nn.add(Layer(2, activation=Relu))
nn.add(Layer(5, activation=Relu))
nn.add(Layer(1, activation=Relu))

X = np.array([[random() for _ in range(2)] for _ in range(1000)])
y = np.array([i[0] + i[1] for i in X])

nn.fit(X, y, epochs=10)

Epoch 1/10:
Error: 0.0364985504131815

Epoch 2/10:
Error: 0.005417132595421775

Epoch 3/10:
Error: 0.0026162492924910845

Epoch 4/10:
Error: 0.0014008464191218222

Epoch 5/10:
Error: 0.0008065960335853603

Epoch 6/10:
Error: 0.0005114180166782833

Epoch 7/10:
Error: 0.00034643069548445947

Epoch 8/10:
Error: 0.00024538778006405465

Epoch 9/10:
Error: 0.00018177276061855506

Epoch 10/10:
Error: 0.00013980369055809048


Finished Training


In [561]:
test_set = np.array([[3, 8], [50, 32],[0.4, 0.4], [0.27, 0.32], [0.1, 0.13], [0.76, 0.22], [0.2, 0.3]]) # 11, 82, 0.8, 0.59, 0.23, 0.98
nn.predict(test_set)

array([[11.86084197, 72.29730881,  0.80161378,  0.58970399,  0.22825553,
         0.99183532,  0.49839215]])

In [562]:
import seaborn as sns

iris = sns.load_dataset('iris')

# we gebruiken .values om een Numpy array te krijgen in plaats van een Pandas DataFrame
X_iris = iris.drop('species', axis=1).values
X_iris = np.array([*X_iris[:50], *X_iris[100:150]])



In [563]:
y_iris = iris['species'].values
y_iris = np.array([*y_iris[:50], *y_iris[100:150]], ndmin=2).T 
# y_iris = y_iris.reshape(-1, 1)


In [564]:
X_iris.shape, y_iris.shape

((100, 4), (100, 1))

In [565]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
encoded_y_iris = le.fit_transform(y_iris)

/Users/aliozcan/miniforge3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [566]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_iris, encoded_y_iris, test_size=0.2, stratify=y_iris, random_state=42)

In [567]:
from random import random
model = NeuralNetwork()

model.add(Layer(4, activation=Relu))
model.add(Layer(7, activation=Relu))
model.add(Layer(1, activation=Sigmoid))

X_train

# X = np.array([[random() for _ in range(2)] for _ in range(1000)])
# y = np.array([[i[0] + i[1]] for i in X])

model.fit(X_train, y_train, epochs=10, loss_function=MSE)

Epoch 1/10:
Error: 0.17960842395927373

Epoch 2/10:
Error: 0.0006412768407971352

Epoch 3/10:
Error: 0.00025289813255518074

Epoch 4/10:
Error: 0.0001446324352972104

Epoch 5/10:
Error: 9.473175642752612e-05

Epoch 6/10:
Error: 6.917426980940955e-05

Epoch 7/10:
Error: 5.157290588180118e-05

Epoch 8/10:
Error: 4.024959485563963e-05

Epoch 9/10:
Error: 3.2140045651976645e-05

Epoch 10/10:
Error: 2.6279689521322828e-05


Finished Training


In [569]:
pred = model.predict(X_test)

print(y_test.tolist())
print([round(x) for x in pred[0]])

rounded_predictions = np.array([round(x) for x in pred[0]])

from sklearn.metrics import accuracy_score

print(f"\naccuracy score: {accuracy_score(y_test, rounded_predictions)}")

[1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1]
[1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1]

accuracy score: 1.0
